# Hash minimal discriminant

Votre système de gestion de sources est en panne et il est devenu impossible de définir des tags.
Vous allez devoir noter dans votre cahier l'identifiant des révisions que vous avez livré ou que vous allez livrer.

Comme copier un SHA-1 intégralement est long et soumis à erreur, vous écrivez un script pour obtenir le plus petit SHA-1 discriminant par commit pour un dépot donné.

Vous avez en entrée la liste de tous les commits avec leur hash :

```
message,hash
foo,d5acd0e29ec0785872bdb17cb07d75d00adc3d2f
bar,417b5f004f35946d9cdc4df18681d962f2b86295
baz,417b5f09e3f48f8f38cb33a8b7ad7ce51a47c72c
qux,d5ac7b961e85aa0ba8913e0ff008cdf2ea6ebbaa
```

En sortie, le message avec le plus petit préfixe qui permet d'identifier ce commit :

```
message,short_hash,hash
foo,d5acd,d5acd0e29ec0785872bdb17cb07d75d00adc3d2f
bar,417b5f00,417b5f004f35946d9cdc4df18681d962f2b86295
baz,417b5f09,417b5f09e3f48f8f38cb33a8b7ad7ce51a47c72c
qux,d5ac7,d5ac7b961e85aa0ba8913e0ff008cdf2ea6ebbaa
```

----

On commence par une fonction qui peut lire les données d'entrée :

In [1]:
import csv
from typing import Dict


def parse_input(text: str) -> Dict[str, str]:
    """
    Construit un dictionnaire: hash -> message de commit
    """
    return {row["hash"]: row["message"] for row in csv.DictReader(text.splitlines())}


sample_data = """message,hash
foo,d5acd0e29ec0785872bdb17cb07d75d00adc3d2f
bar,417b5f004f35946d9cdc4df18681d962f2b86295
baz,417b5f09e3f48f8f38cb33a8b7ad7ce51a47c72c
qux,d5ac7b961e85aa0ba8913e0ff008cdf2ea6ebbaa
"""


commits = parse_input(sample_data)


assert commits == {
    "d5acd0e29ec0785872bdb17cb07d75d00adc3d2f": "foo",
    "417b5f004f35946d9cdc4df18681d962f2b86295": "bar",
    "417b5f09e3f48f8f38cb33a8b7ad7ce51a47c72c": "baz",
    "d5ac7b961e85aa0ba8913e0ff008cdf2ea6ebbaa": "qux",
}

On aura aussi besoin d'une fonction qui calcule le préfixe commun de deux chaînes :

In [2]:
from itertools import takewhile


def common_prefix(s1: str, s2: str) -> str:
    # On itère en parallèle sur les deux chaînes avec zip(),
    # qui nous renvoie une paire de caractères,
    # et on continue tant que les deux sont identiques
    return "".join(
        pair[0] for pair in takewhile(lambda pair: pair[0] == pair[1], zip(s1, s2))
    )


assert common_prefix("417b5f004f35946d9cdc4df", "417b5f09e3f48f8f38cb33a") == "417b5f0"
assert common_prefix("417b5f004f35946d9cdc4df", "d5acd0e29ec0785872bdb17") == ""

On écrit maintenant la fonction qui, à partir d'une liste de hashes, va déterminer pour chacun le préfixe discriminant minimal.

In [3]:
from typing import Iterable


def minimal_prefixes(hashes: Iterable[str]) -> Dict[str, str]:
    """
    Construit un dictionnaire qui associe un préfixe minimal à chaque hash
    """
    prefixes: Dict[str, str] = {}
    for hash_ in hashes:
        add_hash(hash_, prefixes)
    return prefixes


def add_hash(this_hash: str, prefixes: Dict[str, str]) -> None:
    """
    Insère un nouveau hash dans le dictionnaire
    
    Si le hash commence par le même préfixe qu'un autre déjà dans le
    dictionnaire, celui-ci n'est plus discriminant.
    
    Le nouveau préfixe minimal de chacun des deux hash sera le préfixe
    commun concaténé au caractère suivant.
    """
    for other_hash, prefix in prefixes.items():
        if this_hash.startswith(prefix):
            common = common_prefix(this_hash, other_hash)
            prefixes[this_hash] = common + this_hash[len(common)]
            prefixes[other_hash] = common + other_hash[len(common)]
            return
    prefixes[this_hash] = this_hash[0]
    

hashes = commits.keys()


prefixes = minimal_prefixes(hashes)


assert prefixes == {
    "d5acd0e29ec0785872bdb17cb07d75d00adc3d2f": "d5acd",
    "417b5f004f35946d9cdc4df18681d962f2b86295": "417b5f00",
    "417b5f09e3f48f8f38cb33a8b7ad7ce51a47c72c": "417b5f09",
    "d5ac7b961e85aa0ba8913e0ff008cdf2ea6ebbaa": "d5ac7",
}

Et maintenant si on met tout ensemble :

In [4]:
def output(commits: Dict[str, str], prefixes: Dict[str, str]) -> str:
    lines = ["message,short_hash,hash"]
    for hash_, message in commits.items():
        lines.append(f"{hash_},{prefixes[hash_]},{message}")
    return "\n".join(lines)


assert (
    output(commits, prefixes)
    == """message,short_hash,hash
d5acd0e29ec0785872bdb17cb07d75d00adc3d2f,d5acd,foo
417b5f004f35946d9cdc4df18681d962f2b86295,417b5f00,bar
417b5f09e3f48f8f38cb33a8b7ad7ce51a47c72c,417b5f09,baz
d5ac7b961e85aa0ba8913e0ff008cdf2ea6ebbaa,d5ac7,qux"""
)

In [5]:
def main(text: str) -> str:
    commits = parse_input(text)
    prefixes = minimal_prefixes(commits.keys())
    print(output(commits, prefixes))


main(sample_data)

message,short_hash,hash
d5acd0e29ec0785872bdb17cb07d75d00adc3d2f,d5acd,foo
417b5f004f35946d9cdc4df18681d962f2b86295,417b5f00,bar
417b5f09e3f48f8f38cb33a8b7ad7ce51a47c72c,417b5f09,baz
d5ac7b961e85aa0ba8913e0ff008cdf2ea6ebbaa,d5ac7,qux
